In [1]:
import pandas as pd
import numpy as np
import glob
import os

path = "../tasktwo_/submissions/" 
files = glob.glob(os.path.join(path, "*.csv"))
print(f"Найдено {len(files)} файлов для объединения")

Найдено 10 файлов для объединения


In [2]:
def load_submission(file_path):
    weight = int(os.path.basename(file_path).split(".")[0])
    df = pd.read_csv(file_path)
    df_onehot = pd.get_dummies(df['label'], prefix='label')
    df_onehot = df_onehot * weight
    df_onehot['idx'] = df['idx']
    return df_onehot, weight

sample_df, sample_weight = load_submission(files[0])
sample_df.head(), sample_weight

(   label_0  label_1  label_2  label_3  label_4  label_5  idx
 0      122        0        0        0        0        0    1
 1      122        0        0        0        0        0    2
 2      122        0        0        0        0        0    3
 3        0        0        0        0        0      122    4
 4        0        0        0        0        0      122    5,
 122)

In [3]:
all_data = None
total_weights = {}

for file in files:
    df_onehot, weight = load_submission(file)
    total_weights[file] = weight
    
    if all_data is None:
        all_data = df_onehot
    else:
        all_data = all_data.merge(df_onehot, on='idx', how='outer', suffixes=('', '_dup')).fillna(0)
        for col in df_onehot.columns:
            if col != 'idx' and col + '_dup' in all_data.columns:
                all_data[col] += all_data[col + '_dup']
                all_data.drop(columns=[col + '_dup'], inplace=True)

print("Все файлы объединены с учетом весов")
all_data.head()

Все файлы объединены с учетом весов


,label_0,label_1,label_2,label_3,label_4,label_5,idx
0,1140,0,0,0,0,0,1
1,675,126,0,0,0,339,2
2,1140,0,0,0,0,0,3
3,0,0,0,0,220,920,4
4,0,0,0,0,0,1140,5


In [4]:
label_cols = [c for c in all_data.columns if c.startswith('label_')]
all_data[label_cols] = all_data[label_cols].div(all_data[label_cols].sum(axis=1), axis=0)
all_data.head()

,label_0,label_1,label_2,label_3,label_4,label_5,idx
0,1.000000,0.000000,0.0,0.0,0.000000,0.000000,1
1,0.592105,0.110526,0.0,0.0,0.000000,0.297368,2
2,1.000000,0.000000,0.0,0.0,0.000000,0.000000,3
3,0.000000,0.000000,0.0,0.0,0.192982,0.807018,4
4,0.000000,0.000000,0.0,0.0,0.000000,1.000000,5


In [5]:
all_data['label'] = all_data[label_cols].idxmax(axis=1).str.replace('label_', '').astype(int)
ideal_submission = all_data[['idx', 'label']]
ideal_submission.to_csv('ideal_submission.csv', index=False)
print("Идеальный сабмишн сформирован и сохранен в ideal_submission.csv")
ideal_submission.head(10)

Идеальный сабмишн сформирован и сохранен в ideal_submission.csv


,idx,label
0,1,0
1,2,0
2,3,0
3,4,5
4,5,5
5,6,1
6,7,1
7,8,0
8,9,5
9,10,0
